Code for training and backtesting. This code uses the outputs generated by the following programs:
1. CheckFilesOpening_Lsa_summary.ipynb
2. Comparing2CsvFiles.ipynb
3. CombineSentimentTradingData.ipynb

#notes about dates


the dates range from  3rd June 2022 to 16th December 2023 and there are 558 rows in the merged_df dataframe
TRAIN_START_DATE = '2023-01-01'  change to '2022-06-03'
TRAIN_END_DATE = '2023-06-30'    change to '2023-06-30'
TRADE_START_DATE = '2023-07-01'  change to '2023-07-01'
TRADE_END_DATE = '2023-12-31'    change to '2023-12-16'
Data for about 1 year for train and 6 months for trade

In [ ]:
# -------------------------------------------------
# Step 0: Prerequisites & Setup
# -------------------------------------------------
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install the required libraries
!pip install finrl
!pip install yfinance stockstats
!pip install gym==0.21.0
!pip install matplotlib pandas scikit-learn
!pip install ta
!pip install shimmy>=2.0
!pip install gymnasium
!pip install stable_baselines3
!pip install alpaca-trade-api
!pip install exchange_calendars
!pip install wrds
%matplotlib inline

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 46.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=76130eb5cba392abd7d5852f715857d27298a9f0f21fbc1168f8195ef1f97e9e
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/benstaf/FinRL_DeepSeek.git
%cd /content
%cd FinRL_DeepSeek
!pip install pandas numpy yfinance gym stable_baselines3 ta alpaca-trade-api exchange_calendars wrds matplotlib scikit-learn finrl
import sys
sys.path.append('/content/FinRL_DeepSeek')
print(sys.path)

Cloning into 'FinRL_DeepSeek'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 145 (delta 94), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (145/145), 1.21 MiB | 8.24 MiB/s, done.
Resolving deltas: 100% (94/94), done.
/content
/content/FinRL_DeepSeek
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/FinRL_DeepSeek']


In [ ]:
# -------------------------------------------------
# Step 1: Load Sentiment & Stock Price Data
# -------------------------------------------------
import pandas as pd


# Forward-fill sentiment for days with missing data
#merged_df = pd.read_csv('/content/aapl_trading_sentiment_data_all_days.csv')
merged_df = pd.read_csv('/content/aapl_trading_sentiment_data_all_days_Counterfactual.csv')
print(merged_df.head())

                  date   tic       close        high         low        open  \
0  2022-06-03 00:00:00  AAPL  143.429596  147.970001  144.460007  146.899994   
1  2022-06-04 00:00:00  AAPL         NaN         NaN         NaN         NaN   
2  2022-06-05 00:00:00  AAPL         NaN         NaN         NaN         NaN   
3  2022-06-06 00:00:00  AAPL  144.179428  148.570007  144.899994  147.029999   
4  2022-06-07 00:00:00  AAPL  146.714951  149.000000  144.100006  144.350006   

       volume  day      macd     boll_ub  ...     rsi_30     cci_30  \
0  88570300.0  4.0 -2.932397  154.894657  ...  44.333586 -44.829806   
1         NaN  NaN       NaN         NaN  ...        NaN        NaN   
2         NaN  NaN       NaN         NaN  ...        NaN        NaN   
3  71598400.0  0.0 -2.759028  153.013285  ...  44.791651 -35.897958   
4  67808200.0  1.0 -2.389492  152.482277  ...  46.336339 -25.492797   

       dx_30  close_30_sma  close_60_sma        vix  turbulence  Unnamed: 0  \
0  15.563760 

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   558 non-null    object 
 1   tic                    558 non-null    object 
 2   close                  387 non-null    float64
 3   high                   387 non-null    float64
 4   low                    387 non-null    float64
 5   open                   387 non-null    float64
 6   volume                 387 non-null    float64
 7   day                    387 non-null    float64
 8   macd                   387 non-null    float64
 9   boll_ub                387 non-null    float64
 10  boll_lb                387 non-null    float64
 11  rsi_30                 387 non-null    float64
 12  cci_30                 387 non-null    float64
 13  dx_30                  387 non-null    float64
 14  close_30_sma           387 non-null    float64
 15  close_

In [ ]:
#Step 2 — normalizing dates


import pandas as pd

#Normalize data
processed_df = merged_df.copy()
processed_df['date'] = pd.to_datetime(processed_df['date']).dt.normalize()

# Drop unwanted columns like 'Unnamed: 0'
processed_df = processed_df.drop(columns=[col for col in processed_df.columns if 'Unnamed:' in col or col.endswith('_y')])
processed_df.columns = [col.replace('_x', '') for col in processed_df.columns]

#Construct full (date, tic) frame and forward-fill
import itertools

list_ticker = processed_df["tic"].unique().tolist()
list_date = pd.date_range(start=processed_df['date'].min(), end=processed_df['date'].max())
combination = list(itertools.product(list_date, list_ticker))

processed_full = pd.DataFrame(combination, columns=["date", "tic"])
processed_full['date'] = pd.to_datetime(processed_full['date']).dt.normalize()

# Merge and forward fill
processed_full = processed_full.merge(processed_df, on=["date", "tic"], how="left")
processed_full = processed_full.sort_values(by=["tic", "date"]).ffill()

#Split into training and trading sets
from finrl.meta.preprocessor.preprocessors import data_split

TRAIN_START_DATE = '2022-06-03'
TRAIN_END_DATE   = '2023-06-30'
TRADE_START_DATE = '2023-07-01'
TRADE_END_DATE   = '2023-12-16'

train_df = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
trade_df = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)

# Confirm final shape
print("Train shape:", train_df.shape)
print("Trade shape:", trade_df.shape)
print(train_df[['date', 'tic', 'close', 'sentiment']].head())


Train shape: (392, 20)
Trade shape: (168, 20)
        date   tic       close  sentiment
0 2022-06-03  AAPL  143.429596        3.0
1 2022-06-04  AAPL  143.429596        2.0
2 2022-06-05  AAPL  143.429596        3.0
3 2022-06-06  AAPL  144.179428        2.0
4 2022-06-07  AAPL  146.714951        4.0


In [ ]:
#Step 3 — training of an RL Agent using FinRL
import numpy as np
import gymnasium as gym  # Use gymnasium instead of gym
from env_stocktrading import StockTradingEnv
from finrl.config import INDICATORS
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Set the desired state dimension
desired_state_space = 18

# Step i: Initialize the training environment
env = StockTradingEnv(
    df=train_df,                # training DataFrame from Step 2
    stock_dim=1,                # Only one ticker (AAPL)
    num_stock_shares=[100],     # Starting with 100 shares for AAPL
    buy_cost_pct=[0.001],
    sell_cost_pct=[0.001],
    hmax=100,
    initial_amount=1_000_000,
    reward_scaling=1e-4,
    state_space=desired_state_space,  # Intended state dimension
    action_space=1,             # For one stock
    tech_indicator_list=INDICATORS,
    risk_indicator_col='sentiment'
)

# Enable the new step API
if hasattr(env, "new_step_api"):
    env.new_step_api = True

# Step ii: Define a custom observation wrapper using gymnasium
class CustomObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, desired_shape):
        super().__init__(env)
        self.desired_shape = desired_shape
        # Override the observation_space to force a flat vector with the desired shape.
        self.observation_space = gym.spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(desired_shape,),
            dtype=np.float32
        )

    def observation(self, obs):
        # If obs is a dict, attempt to concatenate its flattened values
        if isinstance(obs, dict):
            try:
                obs = np.concatenate([np.ravel(v) for v in obs.values()])
            except Exception as e:
                print("Error concatenating dict values:", e)
                obs = np.array([])
        # If obs is a list/tuple, convert each element to an array and flatten
        elif isinstance(obs, (list, tuple)):
            try:
                obs = np.concatenate([np.array(o, dtype=np.float32).flatten() for o in obs])
            except Exception as e:
                print("Error converting list/tuple to array:", e)
                obs = np.array([])
        else:
            try:
                obs = np.array(obs, dtype=np.float32).flatten()
            except Exception as e:
                print("Error converting observation to array:", e)
                obs = np.array([])

        # Pad with zeros if too short, or truncate if too long
        if len(obs) < self.desired_shape:
            padded = np.zeros(self.desired_shape, dtype=np.float32)
            padded[:len(obs)] = obs
            obs = padded
        elif len(obs) > self.desired_shape:
            obs = obs[:self.desired_shape]
        return obs

# Wrap the environment with the custom observation wrapper
env = CustomObservationWrapper(env, desired_state_space)

# Step iii: Verify the observation shape from reset
obs, info = env.reset()  # Gymnasium reset() returns (obs, info)
print("Observation from env (should have shape {}):".format(desired_state_space))
print(obs, obs.shape)

# Step iv: Wrap the environment in a DummyVecEnv (Stable Baselines3 requires vectorized envs)
env_vec = DummyVecEnv([lambda: env])

# Step v: Initialize and train the PPO agent
model = PPO("MlpPolicy", env_vec, verbose=1)
model.learn(total_timesteps=10000)
trained_model = model
#save trained model
model.save("/content/ppo_finrl_trading_model")
state_space = trained_model.env.observation_space.shape[0]


Observation from env (should have shape 18):
[ 1.0000000e+06  1.4342960e+02  0.0000000e+00 -2.9323971e+00
  1.5489465e+02  1.3339610e+02  4.4333584e+01 -4.4829807e+01
  1.5563760e+01  1.4862311e+02  1.5706927e+02  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00] (18,)
Using cpu device
-----------------------------
| time/              |      |
|    fps             | 800  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
day: 391, episode: 10
begin_total_asset: 1014342.96
end_total_asset: 1160663.20
total_reward: 146320.24
total_cost: 3805.92
total_trades: 387
Sharpe: 1.371
------------------------------------------
| time/                   |              |
|    fps                  | 574          |
|    iterations           | 2            |
|    time_elapsed         | 7            |
|    total_timesteps      | 4096         |
| train/                  |     

#Backtesting - performance metrics and

In [ ]:
#print("Columns in df_account_value:", df_account_value.columns)


In [ ]:
# Step 4 - Backtesting performance metrics (Sharpe Ratio)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import gymnasium as gym  # using gymnasium (make sure it's installed: pip install gymnasium)
from stable_baselines3.common.vec_env import DummyVecEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS
from env_stocktrading import StockTradingEnv  # adjust path as needed

# ================================
# Step 1: Build the Raw Trading Environment for Backtesting
# ================================
# Assume trade_df is your backtesting DataFrame from Step 2.
stock_dim = len(trade_df['tic'].unique())  # Expected to be 1 for AAPL

env_trade_raw = StockTradingEnv(
    df=trade_df,
    stock_dim=stock_dim,
    num_stock_shares=[100] * stock_dim,
    buy_cost_pct=[0.001] * stock_dim,
    sell_cost_pct=[0.001] * stock_dim,
    hmax=100,
    initial_amount=1_000_000,
    reward_scaling=1e-4,
    state_space=18,  # Our target observation dimension
    action_space=stock_dim,
    tech_indicator_list=INDICATORS,
    risk_indicator_col='sentiment'
)

# ================================
# Step 2: Create a Custom Observation Wrapper That Forces a Fixed Observation Shape
# ================================
class CustomObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, desired_shape):
        super(CustomObservationWrapper, self).__init__(env)
        self.desired_shape = desired_shape
        self.observation_space = gym.spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(desired_shape,),
            dtype=np.float32
        )

    def observation(self, obs):
        # Flatten the observation into a 1D array
        obs = np.array(obs, dtype=np.float32).flatten()
        # If too short, pad with zeros; if too long, truncate.
        if len(obs) < self.desired_shape:
            padded = np.zeros(self.desired_shape, dtype=np.float32)
            padded[:len(obs)] = obs
            return padded
        elif len(obs) > self.desired_shape:
            return obs[:self.desired_shape]
        else:
            return obs

    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        return self.observation(obs), info

    def step(self, action):
        # IMPORTANT: Return 5 values (Gymnasium API): (obs, reward, terminated, truncated, info)
        obs, reward, terminated, truncated, info = self.env.step(action)
        return self.observation(obs), reward, terminated, truncated, info

# ================================
# Step 3: Create a Custom Action Wrapper to Force Actions as Arrays
# ================================
from gymnasium import ActionWrapper

class CustomActionWrapper(ActionWrapper):
    def action(self, action):
        return np.atleast_1d(action)

# ================================
# Step 4: Wrap the Raw Environment with Observation and Action Wrappers
# ================================
env_wrapped = CustomObservationWrapper(env_trade_raw, desired_shape=18)
env_wrapped = CustomActionWrapper(env_wrapped)

# ================================
# Step 5: Create a Vectorized Environment Using DummyVecEnv
# ================================
env_trade_vec = DummyVecEnv([lambda: env_wrapped])

# Patch get_sb_env to return the vectorized environment and its reset observation.
env_trade_vec.get_sb_env = lambda: (env_trade_vec, env_trade_vec.reset()[0])

# Attach your trade DataFrame (needed by DRL_prediction)
env_trade_vec.df = trade_df

# (Optional) Check the observation shape from the vectorized environment:
obs = env_trade_vec.reset()[0]
print("Trading env observation shape:", obs.shape)  # Expected: (n_env, 18), e.g., (1, 18)

# ================================
# Step 6: Backtesting with DRL_prediction
# ================================
agent = DRLAgent(env=env_trade_vec)
df_account_value, df_actions = agent.DRL_prediction(
    model=trained_model,  # your previously trained PPO model
    environment=env_trade_vec,
    deterministic=True
)

# ================================
# Step 7: Compute Performance Metrics & Plot Results
# ================================
df_account_value['date'] = pd.to_datetime(df_account_value['date'])
df_account_value = df_account_value.sort_values('date').set_index('date')

# Compute daily returns
df_account_value['daily_return'] = df_account_value['account_value'].pct_change().fillna(0)
mean_return = df_account_value['daily_return'].mean()
std_return = df_account_value['daily_return'].std()
sharpe_ratio = (mean_return / std_return) * np.sqrt(252)
print("Agent Sharpe Ratio:", sharpe_ratio)

# Plot portfolio value over time
plt.figure(figsize=(12, 6))
plt.plot(df_account_value.index, df_account_value['account_value'], label='Agent Portfolio Value')
plt.title("Agent Portfolio Value Over Time")
plt.xlabel("Date")
plt.ylabel("Total Asset Value")
plt.legend()
plt.grid(True)
plt.show()

# ================================
# Optional: Compare Against NASDAQ-100 Benchmark
# ================================
benchmark = yf.download('^NDX', start=df_account_value.index.min().strftime("%Y-%m-%d"),
                        end=df_account_value.index.max().strftime("%Y-%m-%d"))
benchmark = benchmark[['Close']]
benchmark.rename(columns={'Close': 'nasdaq_close'}, inplace=True)
benchmark['benchmark_return'] = benchmark['nasdaq_close'].pct_change().fillna(0)
benchmark['benchmark_cum_return'] = (1 + benchmark['benchmark_return']).cumprod()

df_account_value['agent_cum_return'] = (1 + df_account_value['daily_return']).cumprod()

plt.figure(figsize=(12,6))
plt.plot(df_account_value.index, df_account_value['agent_cum_return'], label='Agent Cumulative Return')
plt.plot(benchmark.index, benchmark['benchmark_cum_return'], label='NASDAQ-100 Cumulative Return', linestyle='--')
plt.title("Cumulative Returns: Agent vs. NASDAQ-100")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.grid(True)
plt.show()

benchmark_sharpe = (benchmark['benchmark_return'].mean() / benchmark['benchmark_return'].std()) * np.sqrt(252)
print("NASDAQ-100 Sharpe Ratio:", benchmark_sharpe)


Trading env observation shape: (18,)


[*********************100%***********************]  1 of 1 completed

hit end!
Agent Sharpe Ratio: 0.6097134281754647
NASDAQ-100 Sharpe Ratio: 1.198449684984351


Additional metrics

In [ ]:
# === Additional Performance Metrics for Agent ===

import numpy as np

# Ensure df_account_value is available and indexed by date
portfolio = df_account_value.copy()

# --- Total Return ---
total_return = portfolio['account_value'].iloc[-1] / portfolio['account_value'].iloc[0] - 1

# --- Annualized Return & Volatility ---
daily_returns = portfolio['daily_return']
annual_return = np.mean(daily_returns) * 252
annual_volatility = np.std(daily_returns) * np.sqrt(252)

# --- Max Drawdown ---
cumulative = (1 + daily_returns).cumprod()
peak = cumulative.cummax()
drawdown = (cumulative - peak) / peak
max_drawdown = drawdown.min()

# --- Calmar Ratio ---
calmar_ratio = annual_return / abs(max_drawdown) if max_drawdown != 0 else np.nan

# --- Win/Loss Ratio ---
wins = (daily_returns > 0).sum()
losses = (daily_returns < 0).sum()
win_loss_ratio = wins / losses if losses != 0 else np.nan

# --- Hit Rate ---
hit_rate = wins / (wins + losses)

# --- Print Results ---
print("\n====== Agent Performance Metrics ======")
print(f"Total Return:         {total_return:.2%}")
print(f"Annual Return:        {annual_return:.2%}")
print(f"Annual Volatility:    {annual_volatility:.2%}")
print(f"Sharpe Ratio:         {sharpe_ratio:.3f}")
print(f"Max Drawdown:         {max_drawdown:.2%}")
print(f"Calmar Ratio:         {calmar_ratio:.3f}")
print(f"Win/Loss Ratio:       {win_loss_ratio:.2f}")
print(f"Hit Rate:             {hit_rate:.2%}")
print("========================================")



====== Agent Performance Metrics ======
Total Return:         1.87%
Annual Return:        2.90%
Annual Volatility:    4.74%
Sharpe Ratio:         0.610
Max Drawdown:         -4.66%
Calmar Ratio:         0.623
Win/Loss Ratio:       0.65
Hit Rate:             39.52%


In [ ]:
# Benchmarking against NASDAQ
# === Benchmark Comparison: Performance Metrics Agent vs NASDAQ-100 ===

import numpy as np

# Agent Metrics
agent = df_account_value.copy()
agent['daily_return'] = agent['account_value'].pct_change().fillna(0)
agent_cum_return = agent['account_value'].iloc[-1] / agent['account_value'].iloc[0] - 1
agent_annual_ret = agent['daily_return'].mean() * 252
agent_annual_vol = agent['daily_return'].std() * np.sqrt(252)
agent_sharpe = (agent_annual_ret / agent_annual_vol) if agent_annual_vol != 0 else np.nan
agent_cum = (1 + agent['daily_return']).cumprod()
agent_peak = agent_cum.cummax()
agent_drawdown = (agent_cum - agent_peak) / agent_peak
agent_max_dd = agent_drawdown.min()
agent_calmar = agent_annual_ret / abs(agent_max_dd) if agent_max_dd != 0 else np.nan
agent_win = (agent['daily_return'] > 0).sum()
agent_loss = (agent['daily_return'] < 0).sum()
agent_hit_rate = agent_win / (agent_win + agent_loss)
agent_wlr = agent_win / agent_loss if agent_loss != 0 else np.nan

# Benchmark Metrics
benchmark = yf.download('^NDX', start=agent.index.min().strftime("%Y-%m-%d"),
                        end=agent.index.max().strftime("%Y-%m-%d"))[['Close']]
benchmark.rename(columns={'Close': 'benchmark'}, inplace=True)
benchmark['benchmark_return'] = benchmark['benchmark'].pct_change().fillna(0)
benchmark_cum_return = benchmark['benchmark'].iloc[-1] / benchmark['benchmark'].iloc[0] - 1
benchmark_annual_ret = benchmark['benchmark_return'].mean() * 252
benchmark_annual_vol = benchmark['benchmark_return'].std() * np.sqrt(252)
benchmark_sharpe = (benchmark_annual_ret / benchmark_annual_vol) if benchmark_annual_vol != 0 else np.nan
benchmark_cum = (1 + benchmark['benchmark_return']).cumprod()
benchmark_peak = benchmark_cum.cummax()
benchmark_drawdown = (benchmark_cum - benchmark_peak) / benchmark_peak
benchmark_max_dd = benchmark_drawdown.min()
benchmark_calmar = benchmark_annual_ret / abs(benchmark_max_dd) if benchmark_max_dd != 0 else np.nan
benchmark_win = (benchmark['benchmark_return'] > 0).sum()
benchmark_loss = (benchmark['benchmark_return'] < 0).sum()
benchmark_hit_rate = benchmark_win / (benchmark_win + benchmark_loss)
benchmark_wlr = benchmark_win / benchmark_loss if benchmark_loss != 0 else np.nan


print(type(agent_cum_return), type(benchmark_cum_return))

# === Print Side-by-Side Metrics ===
print("\n📊 Comparison: Agent vs. NASDAQ-100")
print("-----------------------------------------------------------")
print(f"{'Metric':<25} | {'Agent':>10} | {'NASDAQ-100':>10}")
print("-----------------------------------------------------------")
print(f"{'Total Return':<25} | {float(agent_cum_return):10.2%} | {float(benchmark_cum_return):10.2%}")
print(f"{'Annual Return':<25} | {float(agent_annual_ret):10.2%} | {float(benchmark_annual_ret):10.2%}")
print(f"{'Annual Volatility':<25} | {float(agent_annual_vol):10.2%} | {float(benchmark_annual_vol):10.2%}")
print(f"{'Sharpe Ratio':<25} | {float(agent_sharpe):10.3f} | {benchmark_sharpe:10.3f}")
print(f"{'Max Drawdown':<25} | {float(agent_max_dd):10.2%} | {benchmark_max_dd:10.2%}")
print(f"{'Calmar Ratio':<25} | {float(agent_calmar):10.3f} | {benchmark_calmar:10.3f}")
print(f"{'Win/Loss Ratio':<25} | {float(agent_wlr):10.2f} | {benchmark_wlr:10.2f}")
print(f"{'Hit Rate':<25} | {float(agent_hit_rate):10.2%} | {benchmark_hit_rate:10.2%}")
print("-----------------------------------------------------------")


[*********************100%***********************]  1 of 1 completed

<class 'numpy.float64'> <class 'pandas.core.series.Series'>

📊 Comparison: Agent vs. NASDAQ-100
-----------------------------------------------------------
Metric                    |      Agent | NASDAQ-100
-----------------------------------------------------------
Total Return              |      1.87% |      8.74%
Annual Return             |      2.90% |     19.52%
Annual Volatility         |      4.75% |     16.29%
Sharpe Ratio              |      0.610 |      1.198
Max Drawdown              |     -4.66% |    -10.93%
Calmar Ratio              |      0.623 |      1.786
Win/Loss Ratio            |       0.65 |       1.25
Hit Rate                  |     39.52% |     55.65%
-----------------------------------------------------------



<ipython-input-9-f0631fb64e73>:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"{'Total Return':<25} | {float(agent_cum_return):10.2%} | {float(benchmark_cum_return):10.2%}")


In [ ]:

# ✅ Train A2C agent
from stable_baselines3 import A2C

a2c_model = A2C("MlpPolicy", env_vec, verbose=1)
a2c_model.learn(total_timesteps=10000)
a2c_model.save("/content/a2c_finrl_trading_model")
print("✅ A2C training complete.")


Using cpu device
------------------------------------
| time/                 |          |
|    fps                | 480      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.43    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.07    |
|    std                | 1.01     |
|    value_loss         | 0.0129   |
------------------------------------
day: 391, episode: 30
begin_total_asset: 1014342.96
end_total_asset: 998050.25
total_reward: -16292.71
total_cost: 2620.10
total_trades: 301
Sharpe: -0.834
------------------------------------
| time/                 |          |
|    fps                | 473      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_

In [ ]:

from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from env_stocktrading import StockTradingEnv
import numpy as np
import gymnasium as gym

# --- Step i: Define the same environment kwargs ---
desired_state_space = 18

ensemble_env = StockTradingEnv(
    df=train_df,
    stock_dim=1,
    num_stock_shares=[100],
    buy_cost_pct=[0.001],
    sell_cost_pct=[0.001],
    hmax=100,
    initial_amount=1_000_000,
    reward_scaling=1e-4,
    state_space=desired_state_space,
    action_space=1,
    tech_indicator_list=INDICATORS,
    risk_indicator_col='sentiment'
)

# --- Step ii: Observation wrapper (reuse your same logic) ---
class CustomObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, desired_shape):
        super().__init__(env)
        self.desired_shape = desired_shape
        self.observation_space = gym.spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(desired_shape,),
            dtype=np.float32
        )

    def observation(self, obs):
        if isinstance(obs, dict):
            try:
                obs = np.concatenate([np.ravel(v) for v in obs.values()])
            except:
                obs = np.array([])
        elif isinstance(obs, (list, tuple)):
            try:
                obs = np.concatenate([np.array(o, dtype=np.float32).flatten() for o in obs])
            except:
                obs = np.array([])
        else:
            obs = np.array(obs, dtype=np.float32).flatten()

        if len(obs) < self.desired_shape:
            padded = np.zeros(self.desired_shape, dtype=np.float32)
            padded[:len(obs)] = obs
            obs = padded
        elif len(obs) > self.desired_shape:
            obs = obs[:self.desired_shape]
        return obs

# Wrap training and trading envs
train_env = DummyVecEnv([lambda: CustomObservationWrapper(ensemble_env, desired_state_space)])

# --- Step iii: Train PPO and A2C ---
ppo_model = PPO("MlpPolicy", train_env, verbose=0)
a2c_model = A2C("MlpPolicy", train_env, verbose=0)

ppo_model.learn(total_timesteps=10000)
a2c_model.learn(total_timesteps=10000)

# --- Step iv: Ensemble testing environment ---
test_env = StockTradingEnv(
    df=trade_df,
    stock_dim=1,
    num_stock_shares=[100],
    buy_cost_pct=[0.001],
    sell_cost_pct=[0.001],
    hmax=100,
    initial_amount=1_000_000,
    reward_scaling=1e-4,
    state_space=desired_state_space,
    action_space=1,
    tech_indicator_list=INDICATORS,
    risk_indicator_col='sentiment'
)
test_env = CustomObservationWrapper(test_env, desired_state_space)

# --- Step v: Ensemble evaluation loop ---
obs, info = test_env.reset()
ensemble_rewards = []
done = False

while not done:
    ppo_action, _ = ppo_model.predict(obs)
    a2c_action, _ = a2c_model.predict(obs)

    avg_action = ((ppo_action + a2c_action) / 2).astype(int)
    obs, reward, terminated, truncated, info = test_env.step(avg_action)
    done = terminated or truncated
    ensemble_rewards.append(reward)

print("✅ Ensemble evaluation complete.")
print(f"📈 Total ensemble reward: {np.sum(ensemble_rewards):.2f}")


day: 391, episode: 10
begin_total_asset: 1014342.96
end_total_asset: 1098065.30
total_reward: 83722.34
total_cost: 3815.57
total_trades: 390
Sharpe: 1.169
day: 391, episode: 20
begin_total_asset: 1014342.96
end_total_asset: 1015268.85
total_reward: 925.89
total_cost: 3658.01
total_trades: 388
Sharpe: 0.037
day: 391, episode: 30
begin_total_asset: 1014342.96
end_total_asset: 1011966.24
total_reward: -2376.72
total_cost: 3871.17
total_trades: 386
Sharpe: -0.022
day: 391, episode: 40
begin_total_asset: 1014342.96
end_total_asset: 997380.14
total_reward: -16962.82
total_cost: 3178.13
total_trades: 347
Sharpe: -0.774
day: 391, episode: 50
begin_total_asset: 1014342.96
end_total_asset: 1273983.12
total_reward: 259640.16
total_cost: 1059.75
total_trades: 390
Sharpe: 0.751
✅ Ensemble evaluation complete.
📈 Total ensemble reward: 4.22


In [ ]:

# 📦 Add DDPG to the ensemble
from stable_baselines3 import DDPG

# Train DDPG agent
ddpg_model = DDPG("MlpPolicy", env_vec, verbose=1)
ddpg_model.learn(total_timesteps=10000)
ddpg_model.save("/content/ddpg_finrl_trading_model")
print("✅ DDPG training complete.")

# Load trained agents for ensemble (PPO and A2C already trained)
ppo_model = PPO.load("/content/ppo_finrl_trading_model")
a2c_model = A2C.load("/content/a2c_finrl_trading_model")
ddpg_model = DDPG.load("/content/ddpg_finrl_trading_model")

# Ensemble Evaluation using weighted decision (e.g., PPO:0.5, A2C:0.25, DDPG:0.25)
test_env = StockTradingEnv(
    df=trade_df,
    stock_dim=1,
    num_stock_shares=[100],
    buy_cost_pct=[0.001],
    sell_cost_pct=[0.001],
    hmax=100,
    initial_amount=1_000_000,
    reward_scaling=1e-4,
    state_space=state_space,
    action_space=1,
    tech_indicator_list=INDICATORS,
    risk_indicator_col='sentiment'
)
test_env = CustomObservationWrapper(test_env, desired_state_space)

obs, info = test_env.reset()
ensemble_rewards = []
done = False

while not done:
    ppo_action, _ = ppo_model.predict(obs, deterministic=True)
    a2c_action, _ = a2c_model.predict(obs, deterministic=True)
    ddpg_action, _ = ddpg_model.predict(obs, deterministic=True)

    # Weighted average of actions
    weighted_action = ((0.5 * ppo_action + 0.25 * a2c_action + 0.25 * ddpg_action)).astype(int)

    obs, reward, terminated, truncated, info = test_env.step(weighted_action)
    done = terminated or truncated
    ensemble_rewards.append(reward)

print("✅ DDPG-enhanced ensemble evaluation complete.")
print(f"📈 Total ensemble reward: {np.sum(ensemble_rewards):.2f}")


Using cpu device
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 35       |
|    time_elapsed    | 43       |
|    total_timesteps | 1568     |
| train/             |          |
|    actor_loss      | -930     |
|    critic_loss     | 292      |
|    learning_rate   | 0.001    |
|    n_updates       | 1467     |
---------------------------------
day: 391, episode: 60
begin_total_asset: 1014342.96
end_total_asset: 1293813.59
total_reward: 279470.63
total_cost: 999.00
total_trades: 391
Sharpe: 0.790
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 34       |
|    time_elapsed    | 91       |
|    total_timesteps | 3136     |
| train/             |          |
|    actor_loss      | -647     |
|    critic_loss     | 3.89e+03 |
|    learning_rate   | 0.001    |
|    n_updates       | 3035     |
---------------------------------
------------